In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from gpt_index.composability.joint_qa_summary import QASummaryGraphBuilder
from gpt_index import SimpleDirectoryReader, ServiceContext, LLMPredictor
from gpt_index.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
reader = SimpleDirectoryReader('../paul_graham_essay/data')
documents = reader.load_data()

In [21]:
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context_gpt4 = ServiceContext.from_defaults(llm_predictor=llm_predictor_gpt4, chunk_size_limit=1024)

llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context_chatgpt = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt, chunk_size_limit=1024)

Unknown max input size for gpt-3.5-turbo, using defaults.


In [22]:
graph = QASummaryGraphBuilder().build_graph_from_documents(documents, service_context=service_context)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 20729 tokens
> [build_index_from_nodes] Total embedding token usage: 20729 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage:

In [7]:
graph.save_to_disk('test_qa_summary_graph.json')

In [23]:
graph = ComposableGraph.load_from_disk('test_qa_summary_graph.json')

In [24]:
# set query config
query_configs = [
    {
        "index_struct_type": "simple_dict",
        "query_mode": "default",
        "query_kwargs": {
            "similarity_top_k": 1
        },
    },
    {
        "index_struct_type": "list",
        "query_mode": "default",
        "query_kwargs": {
            "response_mode": "tree_summarize",
            "use_async": True,
            "verbose": True
        },
    },
    {
        "index_struct_type": "tree",
        "query_mode": "default",
        "query_kwargs": {
            "verbose": True
        },
    },
]

In [25]:
len(graph._docstore.docs)

50

In [14]:
response = graph.query(
    "Can you give me a summary of the author's life?", 
    query_configs=query_configs, 
    service_context=service_context_gpt4
)

>[Level 0] Current response: ANSWER: 2
The chosen summary is "Use this index for summarization queries," as the question explicitly asks for a summary of the author's life. A summarization query requires the generation of a brief summary that contains the most important information about the author's life, and this option is specifically designed to facilitate such queries. On the other hand, option 1 is more suitable for queries that require the retrieval of specific context from documents, which is not applicable to the given question.
INFO:gpt_index.indices.query.tree.leaf_query:>[Level 0] Selected node: [2]/[2]
>[Level 0] Selected node: [2]/[2]
>[Level 0] Selected node: [2]/[2]
>[Level 0] Node [2] Summary text: Use this index for summarization queries
> Got node text: 		

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supp...
> Got node text:

In [15]:
print(response)

The author has experience in programming and co-founding startup accelerators, particularly Y Combinator. They have also pursued interests in painting and writing essays, and have lived in multiple countries including England and Italy. The author reflects on their past experiences to inform their future decisions and has discussed the concept of invention versus discovery in programming languages.


In [26]:
response = graph.query(
    "What did the author do growing up?", 
    query_configs=query_configs,
    service_context=service_context_gpt4
)

>[Level 0] Current response: ANSWER: 1

This summary was selected because the question asks for specific context from the author's life (what they did growing up), and the first choice mentions retrieval of specific context from documents. The second option relates to summarization and is not as relevant to the question.
INFO:gpt_index.indices.query.tree.leaf_query:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Node [1] Summary text: Use this index for queries that require retrieval of specific context from documents.
> Got node text: Growing up, the author mainly worked on writing short stories and programming. They started programming on an IBM 1401 at school, using an early version of Fortran. Later on, they got a TRS-80 micr...


In [27]:
print(response)

Growing up, the author mainly worked on writing short stories and programming. They started programming on an IBM 1401 at school, using an early version of Fortran, and later wrote simple games, a program to predict model rocket flights, and a word processor on a TRS-80 microcomputer. The author also had an interest in philosophy.


In [35]:
response = graph.query(
    "What did the author do during his time in art school?", 
    query_configs=query_configs,
    service_context=service_context_gpt4
)

>[Level 0] Current response: ANSWER: 1

The first choice is more relevant to the question because it mentions retrieval of specific context from documents. The question asks about a specific event or experience (the author's time in art school), making choice 1 better suited to provide the necessary information. Choice 2, on the other hand, is related to summarization which may not give detailed information about the author's time in art school.
INFO:gpt_index.indices.query.tree.leaf_query:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Node [1] Summary text: Use this index for queries that require retrieval of specific context from documents.
> Got node text: The context information does not provide specific details about what the author did during his time in art school. However, it does mention that he took the foundation classes in fundamental subjec...


In [36]:
print(response)

During his time in art school, the author took foundation classes in fundamental subjects like drawing, color, and design.
